# Classification Analysis
This notebook will contain classification analysis for both the sensed and pipelined algorithms. Analysis will be preformed in regards for the sensed and pipelined algorthms themselves, as well as the ensemble algorithms. The analysis for the ensemble algorithm will focus on the HAMF android phones and the HAHF iOS phones.

## Dependencies

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import emeval.viz.geojson as ezgj

In [ ]:
import emeval.metrics.segmentation as ems

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.6f}'.format
import arrow
import numpy as np

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# For maps
import folium
import branca.element as bre

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
import arrow

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

### Get the sensed data for each trip

In [ ]:
%%capture
ems.fill_sensed_section_ranges(pv_la)
ems.fill_sensed_section_ranges(pv_sj)
ems.fill_sensed_section_ranges(pv_ucb)

## Get sensed timeline

```python
def get_trip_ss_and_gts_timeline(pv):
    """
    Get the sensed and ground truth timeline for each evaluation trip range for a given phone view.
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.
    arg2: os
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg3:
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'

    -------
    Returns
    -------
    list
        A list of trips for each phone view. Each trip has two entries, the sensed mode timeline and the ground truth timeline for the corresponding evaluation trip range.
    """
    ...
```

`TODO` break up the timelines by os and accuracy/frequency

In [ ]:
def get_trip_ss_and_gts_timeline(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    trips = []
    for phone_os, phone_map in pv.map().items():
        if os != phone_os:
            continue
        for phone_label, phone_detail_map in phone_map.items():
            if "control" in phone_detail_map["role"]:
#                 print("Ignoring %s phone %s since they are always on" % (phone_detail_map["role"], phone_label))
                continue
            # this spec does not have any calibration ranges, but evaluation ranges are actually cooler
            for r in phone_detail_map["evaluation_ranges"]:
                if r['eval_role_base'] != role:
                    continue
                for tr in r["evaluation_trip_ranges"]:
                    tr_ss  = []
                    tr_gts = []
                    for ss in tr["sensed_section_ranges"]:
                        tr_ss.append(ss)
                    for section in tr["evaluation_section_ranges"]:
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr['trip_id_base'],
                                                                                  section['trip_id_base'],
                                                                                  tr['start_ts'],
                                                                                  tr['end_ts'])
                        if section_gt_leg["type"] == "WAITING":
#                             print("Skipping WAITING section %s %s with potential partway transitions" %
#                                   (tr["trip_id"], section["trip_id"]))
                            continue
                        # this calulcates the metric for the mode

                        ## and now we have the gt mode!
                        gts = {'start_ts': section['start_ts'], 
                               'end_ts': section['end_ts'], 
                               'mode': section_gt_leg['mode']}
                        tr_gts.append(gts)
                # now, we build a timeline for each trip
                trip = tr.copy()
                trip['ss_timeline']  = tr_ss
                trip['gts_timeline'] = tr_gts
                trips.append(trip)
    return trips

## Binary Classification
```python
def get_binary_class(pv, os, role):
    """
    This function computes binary classifications for a given set of trips.
    Using one unit of duration as our base unit, we calculate the following classifications:
        * True Positive
            + A true positive is when we sense that we are in a mode and we are in that mode.
        * False Positive
            + A false positive is when we sense that we are in a mode that we are not it.
        * False Negative
            + A false negative is when we are in a mode, but we do not sense being in that mode.
     
     Note that we compute the binary classifications for each sensed mode, but we combine the 'WALKING' and 'RUNNING' modes.
     Additionally, note that we have use the ground truth base mode when determining hits and misses. 
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.
    arg2: os
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg3:
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'
    
    -------
    Returns
    -------
    list
        A list with the following entries
            [0] A dictionary of true positives with sensed modes as keys and TP hits has values.
            [1] A dictionary of false positives with sensed modes as keys and FP hits has values.
            [2] A dictionary of false negatives with sensed modes as keys and FN hits has values.
    
    """
    ...
```

In [ ]:
def get_binary_class(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    BASE_MODE = {"WALKING": "WALKING",
        "BICYCLING": "CYCLING",
        "ESCOOTER": "CYCLING",
        "BUS": "AUTOMOTIVE",
        "TRAIN": "AUTOMOTIVE",
        "LIGHT_RAIL": "AUTOMOTIVE",
        "SUBWAY": "AUTOMOTIVE",
        "CAR": "AUTOMOTIVE"}
    trips = get_trip_ss_and_gts_timeline(pv, os, role)
    TP = {}
    FP = {}
    FN = {}
    ## loop though all of the trips
    for trip in trips:
        ## when we sense a mode. Used for TP and FP
        for ss in trip['sensed_section_ranges']:
            # sensed duration := TP + FP
            ss_dur  = ss['end_ts'] - ss['start_ts']
            time_in_gt_modes = {}
            gts_dur = 0
            for gts in trip['gts_timeline']:
                # if the gts starts before the ss ends and ends after the ss starts
                if gts['start_ts'] <= ss['end_ts'] and gts['end_ts'] >= ss['start_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    gts_dur += dur
                    if BASE_MODE[gts['mode']] == ss['mode']:
                        # we sense that we are in a mode, and we are
                        TP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
                    else:
                        # we sense that we are in a mode, and we are not
                        FP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
            ## Take care of when we have no gts, which is a FP as we sense we are in a mode, but in fact their is no gt, so we are not
            leftover = ss_dur - gts_dur
            assert leftover >= 0
            FP[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + leftover # FIXME: this gives mode error
        ## Now we check the ground truth timeline, for FN
        for gts in trip['gts_timeline']:
            gts_dur = gts['end_ts'] - gts['start_ts']
            ss_dur = 0
            for ss in trip['sensed_section_ranges']:
                # if the ss starts before gts ends and ss ends after the gts starts
                if ss['start_ts'] <= gts['end_ts'] and ss['end_ts'] >= gts['start_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    ss_dur += dur
                    if BASE_MODE[gts['mode']] != ss['mode']:
                        # we sense that we are not in a mode, but we are
                        FN[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + dur
            leftover = gts_dur - ss_dur
            assert leftover >= 0
            # we sense that we are not in a mode, but we are
            FN[ss['mode']] = time_in_gt_modes.setdefault(ss['mode'], 0) + leftover
    return [TP, FP, FN]

# $F_\beta$ score
$$
F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
$$

```python
def get_F_score(pv, os, role, beta=1):
    """
    This function calculates the F score
    $$
    F_\beta = \frac {(1 + \beta^2) \cdot \mathrm{true\ positive} }{(1 + \beta^2) \cdot \mathrm{true\ positive} + \beta^2 \cdot \mathrm{false\ negative} + \mathrm{false\ positive}}
    $$
    based off data from a given set of phone views. Calls the get binary classification function
    
    ----------
    Parameters
    ----------
    arg1: phone view
        A phone view to recieve timelines for.
    arg2: os
        a phone os to evaluate, must be one of 'ios' or 'android'
    arg3:
        an acuracy/frequency combination, must be one of 'accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'
    arg4: int
        The beta value in which to use in the $F_\beta$ score. Defaults to 1.
        
    -------
    Returns
    -------
    dict:
        A dictionary with sensed modes as the keys and the corresponding of $F_\beta$ scores as the values.
    
    """
    ...
```

In [ ]:
(TP, FP, FN) = get_binary_class(pv_la, 'android', 'HAMFDC')
print(TP, '\n', FP, '\n', FN)

In [ ]:
def get_F_score(pv, os, role, beta=1):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    F_score = {}
    (TP, FP, FN) = get_binary_class(pv, os, role)
    for mode in TP.keys():
        numerator   = (1 + beta**2) * TP[mode]
        denominator = (1+beta**2) * TP[mode] + beta**2*FN[mode] + FP[mode]
        F_score[mode] = (numerator)/(denominator)
    return F_score

In [ ]:
get_F_score(pv_la, 'ios', 'HAHFDC') ## TODO: investigate low walking scores

## Confusion Matrix
We will now generate confusion matrices based off OS and role, with the acctual modes as the rows, the predicted modes as the columns, and the entries as the base unit for the duration measurement

In [ ]:
def get_confusion_matrix(pv, os, role):
    assert os in ['android', 'ios'], 'UNKNOWN OS'
    assert role in ['accuracy_control', 'HAHFDC', 'HAMFDC', 'MAHFDC', 'power_control'], "UNKNOWN ROLE"
    trips = get_trip_ss_and_gts_timeline(pv, os, role)
    ss_mode_v_gt_mode_l = []
    for trip in trips:
        probabilities = {}
        for ss in trip['sensed_section_ranges']:
            ss_dur  = ss['end_ts'] - ss['start_ts']
            time_in_gt_modes = {}
            for i, gts in enumerate(trip['gts_timeline']): # TODO: optimize here
                if gts['end_ts'] >= ss['start_ts'] and gts['start_ts'] <= ss['end_ts']:
                    dur     = min(ss['end_ts'], gts['end_ts']) - max(ss['start_ts'], gts['start_ts'])
                    time_in_gt_modes[gts['mode']] = time_in_gt_modes.setdefault(gts['mode'], 0) + dur
            time_in_gt_modes['NO GT'] = max(ss_dur - sum(time_in_gt_modes.values(), 0.0),0) #TODO: make sure this is right
            for mode in time_in_gt_modes.keys():
                time_in_gt_modes[mode] /= ss_dur
            # TODO: assert sum of all rows == 1 +- epsilon
            time_in_gt_modes['sensed_mode'] = ss['mode']
            ss_mode_v_gt_mode_l.append(time_in_gt_modes)
    return ss_mode_v_gt_mode_l

`TODO` make sure we can get confusion matrix accross phone views

In [ ]:
import itertools
def plot_confusion_matrix(pv, os, role):
    df = pd.DataFrame(get_confusion_matrix(pv_la, os, role)).groupby('sensed_mode').sum()
    fig, ax = plt.subplots(1,1, figsize=(10,10))
    cm = ax.imshow(df, interpolation='nearest',  cmap=plt.cm.Blues)
    ax.set_title(f"Confusion Matrix\n SPEC ID={pv_la.spec_details.CURR_SPEC_ID}\n os={os}\n role={role}")
    plt.colorbar(cm, ax=ax)
    tick_marks = np.arange(len(df))
    ax.set_xticks(np.arange(len(df.columns)))
    ax.set_yticks(np.arange(len(df)))
    ax.set_xticklabels(df)
    ax.set_yticklabels(df.index)
    ax.set_xlabel('True label')
    ax.set_ylabel('Predicted label')

`TODO` plot andoid and ios for all the different a/f configurations

In [ ]:
plot_confusion_matrix(pv_la, 'ios', 'HAHFDC')